In [1]:
!mkdir data
!cp "/content/drive/MyDrive/bci_proj/s2_1.zip" "/content/data"
!unzip -n -q "/content/data/s2_1.zip" -x / -d "/content/data"

In [ ]:
!pip install mne
!pip install -U scikit-learn
!pip install pytorch-lightning
!pip install comet-ml
!pip install local-attention
!pip install einops

In [1]:
from depen import *

In [2]:
from MI import MI_csp_lda

In [5]:
path = "/content/data/sess01_subj02_EEG_MI.mat"
mat_train, mat_test = get_mat_file(path)

m = MI_csp_lda()

In [ ]:
m.train_on_data(mat_train)

In [ ]:
m.check_data(mat_test)

In [ ]:
###############DEEP LEARNING###########

In [2]:
from hparams import hyperparams
hyperparams.path_to_files

'/content/data'

In [3]:
from model import BCI_model_pl
from datasets import BCI_pl_dataset

from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [4]:
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
            ckpt_path = os.path.join(trainer.checkpoint_callback.dirpath, filename)
            trainer.save_checkpoint(ckpt_path)

In [5]:
seed_e(42)

In [6]:
comet_logger = CometLogger(
    save_dir='/content/log/',
    api_key="23CU99n7TeyZdPeegNDlQ5aHf",
    project_name="bci",
    workspace="etzelkut",
    # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
    # experiment_name="default" # Optional
)

CometLogger will be initialized in online mode


In [7]:
dataset_pl = BCI_pl_dataset(hyperparams)
dataset_pl.prepare_data()
dataset_pl.setup()
train_loader = dataset_pl.train_dataloader()
steps_per_epoch = int(len(train_loader))
print(steps_per_epoch)

gg
MERGING !!!!!!!!!
eeg shape:  (100, 62, 4000)
Adding metadata with 2 columns
Replacing existing metadata with 2 columns
100 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
eeg shape:  (100, 62, 4000)
Adding metadata with 2 columns
Replacing existing metadata with 2 columns
100 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
(1980, 62, 4000)
eeg shape:  (1980, 62, 4000)
Adding metadata with 2 columns
Replacing existing metadata with 2 columns
1980 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
(2160, 62, 4000)
eeg shape:  (2160, 62, 4000)
Adding metadata with 2 columns
Replacing existing metadata with 2 columns
2160 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
eeg shape:  (100, 62, 4000)
Adding metadata with 2 columns
Replacing existing metadata with 2 columns
100

In [8]:
checkpoint_callback = ModelCheckpoint(
                                      monitor='val_loss',
                                      save_last=True, 
                                      dirpath= "/content/drive/MyDrive/bci_proj/gg",#os.path.join(path, "/checkpoints"),
                                      filename='samplemodel{epoch}.ckpt',
                                      save_top_k=3,
                                      mode='min',
                                      )
every_epoch = CheckpointEveryNSteps(save_step_frequency = steps_per_epoch, use_modelcheckpoint_filename = True)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Checkpoint directory /content/drive/MyDrive/bci_proj/gg exists and is not empty. With save_top_k=3, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


In [9]:
model = BCI_model_pl(hyperparams, steps_per_epoch = steps_per_epoch)

softmax
glu


In [10]:
trainer = Trainer(callbacks=[checkpoint_callback, every_epoch], #lr_monitor],
                    logger=comet_logger,
                    gpus=1,
                    profiler=True,
                    #auto_lr_find=True, #set hparams
                    #gradient_clip_val=0.5,
                    check_val_every_n_epoch=2,
                    #early_stop_callback=True,
                    max_epochs = hyperparams.epochs,
                    #min_epochs=400,
                    progress_bar_refresh_rate = 0,
                    deterministic=True,)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model, dataset_pl)

In [ ]:
#100, 62, 400

In [ ]:
trainer.test()

In [18]:
checkpoint_name = os.path.join("/content/drive/MyDrive/bci_proj/gg", 'manual_save' + str(228) + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [11]:
model = BCI_model_pl.load_from_checkpoint("/content/drive/MyDrive/bci_proj/gg/manual_save228.ckpt")

softmax
glu


In [87]:
trainer.test(model, dataset_pl.train_dataloader())

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/etzelkut/bci/9bededef00a5439399ffbe18024f2563

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(96.2500, device='cuda:0'),
 'test_loss': tensor(0.2191, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 96.25, 'test_loss': 0.2190903127193451}]

In [29]:
def data_get(train_dataloader):
  x, y, z = torch.zeros((len(train_dataloader) * 64, 62, 400)), torch.zeros((len(train_dataloader) * 64)), torch.zeros((len(train_dataloader) * 64))
  i = 0
  for batch in train_dataloader:
    xx, yy, zz = batch
    i_top = len(xx)
    print(xx.shape, i, i+i_top)
    x[i:i+i_top] = xx
    y[i:i+i_top] = yy
    z[i:i+i_top] = zz
    i += i_top
  x, y, z = x[:i], y[:i], z[:i]
  return x, y, z

In [37]:
from MI import clean_data, preproc_data, csp_time_window, choose_csp_filters, forward_csp
def get_data_proccesed(mi_train, mi_train_label, sfrec = 100, ):
  channels = np.array(['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'FC5', 'FC1', 'FC2',
       'FC6', 'T7', 'C3', 'Cz', 'C4', 'T8', 'TP9', 'CP5', 'CP1', 'CP2',
       'CP6', 'TP10', 'P7', 'P3', 'Pz', 'P4', 'P8', 'PO9', 'O1', 'Oz',
       'O2', 'PO10', 'FC3', 'FC4', 'C5', 'C1', 'C2', 'C6', 'CP3', 'CPz',
       'CP4', 'P1', 'P2', 'POz', 'FT9', 'FTT9h', 'TTP7h', 'TP7', 'TPP9h',
       'FT10', 'FTT10h', 'TPP8h', 'TP8', 'TPP10h', 'F9', 'F10', 'AF7',
       'AF3', 'AF4', 'AF8', 'PO3', 'PO4'], dtype=object)
  eeg_data = mi_train
  print("eeg shape: ", eeg_data.shape)
  labels = mi_train_label # 0 - right, 1 - left # 0 - up, 1 - left, 2 - right, 3 - down
  ch_names = channels.tolist()
  ch_types = ['eeg'] * len(channels)
  info = mne.create_info(ch_names=ch_names, sfreq=sfrec, ch_types=ch_types)
  #
  label_metadata = {'ids': labels.tolist(),}

  df = pd.DataFrame(label_metadata, columns = ['ids'])
  epochs_data = mne.EpochsArray(eeg_data, info=info, metadata=df)
  return epochs_data

class MI_csp_lda_proccesed():
  def __init__(self):
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
    self.lda = LinearDiscriminantAnalysis(solver='eigen', shrinkage=0.25)
  
  def train_on_data(self, mi_train, mi_train_label):
    data = get_data_proccesed(mi_train, mi_train_label)
    data = clean_data(data, new_rate=None)
    #plot_spectral(data)
    data = preproc_data(data)
    #plot_spectral(data, left = 'cyan', right= 'pink')    

    d, W = csp_time_window(data)
    self.csp = choose_csp_filters(W)

    var_left = forward_csp(data["ids == 1"], self.csp)
    var_right = forward_csp(data["ids == 0"], self.csp)

    data_points = np.concatenate((var_left, var_right), axis = 0)
    labels = [1] * len(var_left) + [0] * len(var_right)
    print(labels)
    self.lda.fit(data_points, labels)

    print(self.lda.predict([var_right[0]]))
  
  def check_data(self, mi_train, mi_train_label):
    
    data = get_data_proccesed(mi_train, mi_train_label)
    x = clean_data(data, new_rate=None)

    #plot_spectral(x) #left - blue, right - red

    x = preproc_data(x)
    #plot_spectral(x, left = 'cyan', right= 'pink')

    labels = x.metadata['ids'].tolist()

    var = forward_csp(x, self.csp)

    label_list = self.lda.predict(var)
    acc = self.lda.score(var, labels)

    print(
      "label_list: ",
      label_list, "\n accuracy: ", 
      acc,
    )

    return acc, label_list

In [55]:
x, y, z = data_get(dataset_pl.train_dataloader())
x, y, z = x.numpy(), y.numpy(), z.numpy()
mi_train = x[np.where(y == 0)]
mi_train_label = z[np.where(y == 0)]

torch.Size([64, 62, 400]) 0 64
torch.Size([64, 62, 400]) 64 128
torch.Size([64, 62, 400]) 128 192
torch.Size([64, 62, 400]) 192 256
torch.Size([64, 62, 400]) 256 320
torch.Size([64, 62, 400]) 320 384
torch.Size([64, 62, 400]) 384 448
torch.Size([64, 62, 400]) 448 512
torch.Size([64, 62, 400]) 512 576
torch.Size([64, 62, 400]) 576 640


In [39]:
mi_m = MI_csp_lda_proccesed()

In [40]:
mi_m.train_on_data(mi_train, mi_train_label)

eeg shape:  (160, 62, 400)
Adding metadata with 1 columns
Replacing existing metadata with 1 columns
160 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
new data shape:  (160, 20, 400)
Setting up band-pass filter from 8 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 8.00, 30.00 Hz: -6.02, -6.02 dB

checking eigenvector  0
0.3775155855195199
0.6224349868053698
csp shape:  (20, 4)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [71]:
xt, yt, zt = data_get(dataset_pl.test_dataloader())
xt, yt, zt = xt.numpy(), yt.numpy(), zt.numpy()
mi_test = xt[np.where(yt == 0)]
mi_test_label = zt[np.where(yt == 0)]

torch.Size([64, 62, 400]) 0 64
torch.Size([16, 62, 400]) 64 80


In [42]:
mi_m.check_data(mi_test, mi_test_label)

eeg shape:  (23, 62, 400)
Adding metadata with 1 columns
Replacing existing metadata with 1 columns
23 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
new data shape:  (23, 20, 400)
Setting up band-pass filter from 8 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 8.00, 30.00 Hz: -6.02, -6.02 dB

label_list:  [1 1 1 1 1 1 0 0 1 1 0 1 1 0 0 0 1 0 0 1 1 1 0] 
 accuracy:  0.9130434782608695


(0.9130434782608695,
 array([1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        0]))

In [57]:
def leave_occipital_ch(eeg_data):
  eeg_channels_indices = [22, 23, 24, 25, 26, 27, 31, 28, 29]
  
  eeg_data_occipital = np.empty((10, eeg_data.shape[0], eeg_data.shape[2]))
  eeg_data = np.transpose(eeg_data, (1, 0, 2))
  for i, channel in enumerate(eeg_channels_indices, start=0):
    eeg_data_occipital[i] = eeg_data[channel]
  
  return eeg_data_occipital

In [78]:
from sklearn.cross_decomposition import CCA
import math

def SSVEP_CCA(eeg_data):

  T = np.arange(start=0, stop=4, step=1/100)

  trials = eeg_data.shape[0]

  freqs = [12, 8.57, 6.67, 5.45]
  pred_freq_score = np.empty(trials)
  pred_freq_class = np.empty(trials, dtype=np.int8)

  X = np.transpose(eeg_data, (0,2,1))
  
  for x_index, x in enumerate(X, start=0):
    Y = np.empty((4, len(T), 4))
    for i in range(4):
      for t_index, t in enumerate(T, start=0):
        ref = 2*math.pi*freqs[i]*t
        Y[i][t_index] = [math.sin(ref), math.cos(ref), math.sin(ref*2), math.cos(ref*2)]
      
    cca = CCA()
    scores = np.empty(4)
    for y_index, y in enumerate(Y, start=0):
      cca.fit(x, y)
      scores[y_index] = cca.score(x, y)
    
    pred_freq_score[x_index] = scores.max()
    pred_freq_class[x_index] = scores.argmax() + 1
  
  return pred_freq_class

In [79]:
#[0] * len(mi_test) + [1] * len(erp_test) + [2] * len(ssvep_test)
mi_train_ss = xt[np.where(yt == 2)]
mi_train_ss_label = zt[np.where(yt == 2)]

In [80]:
eeg = leave_occipital_ch(mi_train_ss)

In [81]:
np.transpose(eeg, (1, 0, 2)).shape

(22, 10, 400)

In [82]:
ll = SSVEP_CCA(np.transpose(eeg, (1, 0, 2)))

In [84]:
mi_train_ss_label + 1

array([3., 3., 2., 2., 1., 2., 4., 4., 2., 2., 3., 1., 3., 2., 2., 3., 1.,
       1., 4., 4., 1., 3.], dtype=float32)

In [83]:
ll

array([3, 3, 2, 2, 1, 2, 4, 4, 2, 2, 3, 1, 3, 2, 2, 3, 1, 1, 4, 4, 1, 3],
      dtype=int8)

In [86]:
np.sum(ll == mi_train_ss_label + 1)/len(ll)

1.0

In [49]:
mi_train_ss.shape, eeg.shape

((156, 62, 400), (10, 156, 400))

In [ ]:
#[0] * len(mi_train) + [1] * len(erp_train) + [2] * len(ssvep_train)

In [ ]:
#The stimulus-time interval was set to 80 ms, and the inter-stimulusinterval (ISI) to 135 ms. 
#A single iteration of stimulus presenta-tion in all rows and columns was considered a sequence. 
#There-fore, one sequence consisted of 12 stimulus flashes. A maximumof five sequences (i.e., 60 flashes) was 
#allotted without prolongedinter-sequence intervals for each target character. After the endof five sequences, 
#4.5 s were given to the user for identifying, lo-cating, and gazing at the next target character.

#The offline EEG data that were acquired in the training phase were band-pass filtered between 0.5 and 40 Hz 
#with a 5th order Butterworthdigital filter. The continuous EEG data were segmented from– 200 to 800 ms 
#with respect to stimulus onset and baseline-corrected by subtracting the mean amplitudes in the –200 to 0 ms pre-stimulus interval.

In [ ]:
#(4000, 100, 62)
#(1434800, 62)
#(1, 100)
#array([[1000]], dtype=uint16)
#array([[2, 1, 2, 2, 1, ... ]]  --- (1, 100)
#[[0, 1, 0 ... --- (2, 100)
#array([[array(['left'], dtype='<U4'), array(['right'], dtype='<U5'), ...  --- (1, 100)
#77777777777777777777777777777777777777777777777777777777777777777777777
#array([[array(['1'], dtype='<U1'), array(['right'], dtype='<U5')],
#       [array(['2'], dtype='<U1'), array(['left'], dtype='<U4')]],
#88888888888888888888888888888888888888888888888888888888888888888888888
# array([[array(['Fp1'], dtype='<U3'), array(['Fp2'], dtype='<U3'), ...   --- (1, 62)
# array([[   1,    2,    3, ..., 3998, 3999, 4000]], dtype=uint16)
# array([[   0, 4000]], dtype=uint16)
# (1434800, 4)
# (1, 4)
#121212121212121212121212121212121212121212121212121212121212121212121212
#array([[array(['EMG1'], dtype='<U4'), array(['EMG2'], dtype='<U4'),
#        array(['EMG3'], dtype='<U4'), array(['EMG4'], dtype='<U4')]],
#131313131313131313131313131313131313131313131313131313131313131313131313
#(60000, 62)
#(60000, 62)
#array([[    0, 60000]], dtype=uint16)
#
# sampling rate of 1,000 Hz
# 62 Ag/AgCl electrodes --- 62 EEG and 4 EMG recording electrode
# the first 3 s of each trial beganwith a black fixation
# appropriate hand for 4 s
# blank for 6 s
# each phase had 100 trials
# During the online test phase, the fixation cross  moved right or left, according to the real-time classifier output of the EEG signal 
# we  visualized  the  physiological  brain  responses BCI paradigms: event-related desynchroniza-tion/synchronization (ERD/ERS) for MI
# All EEG data were commonly down-sampled to 100 Hz.
#
# MI data, 20 electrodes in the motor cortex region were selected (FC-5/3/1/2/4/6, C-5/3/1/z/2/4/6, and CP-5/3/1/z/2/4/6).
# The offline EEG data were band-pass filtered between 8 and 30 Hz with a 5th order Butterworth band-pass filter digital filter
# The continuous EEG data were then segmented from 1,000 to 3,500 ms with re-spect to stimulus onset.
# EEG epochs were therefore constitutedas 250 (data points) ×20 (electrodes) ×100 (trials).
# CSPs were used to maximize the discrimina-tion of the binary class
# log-variance features were cal-culated
# The LDA classifier was then calculated to decode theleft- or right-hand imagery task
#
# A subset of the top and bottomtwo rows from the CSP projection matrix and the LDA parame-ters were fed to the online data analysis
# During the online test phase, a sliding window (length, 1.5 s;step size, 0.5 s) was created to classify the real-time EEG data.
# the data in this window buffer were filtered with thefrequency range used in the training phase
# and the CSP projec-tion matrix w was applied to these EEG data
# The LDA outputswere calculated every 0.5 s
# and transformed into coordinatesfor the horizontal x-axis of the cross to provide real-time visual feedback
# MI performance was validated basedon 10 repetitions of 10-fold cross-validation from all 
# MI data (i.e.,training+test data) with the CSP method (CSP-cv)

"""
for i in range(len(mat['EEG_MI_test'][0][0])):
  print(mat['EEG_MI_test'][0][0][i])
  print("########shape########")
  print(mat['EEG_MI_test'][0][0][i].shape)
  print("###################################################")
  print("####", i+1, "####")

"""